# BlazingSQL + cuML NYC Taxi Cab Fare Prediction

This demo uses pubically availible [NYC Taxi Cab Data](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction) to predict the total fare of a taxi ride in New York City given the pickup and dropoff locations. 

In this notebook, we will cover: 
- How to set up [BlazingSQL](https://blazingsql.com) and the [RAPIDS AI](https://rapids.ai/) suite.
- How to read and query csv files with cuDF and BlazingSQL.
- How to implement a linear regression model with cuML.

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-39814657-5&cid=555&t=event&ec=guides&ea=taxi_fare_prediction&dt=taxi_fare_prediction)


## Setup
### Environment Sanity Check 

RAPIDS packages (BlazingSQL included) require Pascal+ architecture to run. For Colab, this translates to a T4 GPU instance. 

The cell below will let you know what type of GPU you've been allocated, and how to proceed.

In [1]:
# tag specs
colab_smi = !nvidia-smi

# focus GPU type
try:
    my_gpu = ' '.join(colab_smi[7].split()[2:4])
# not on gpu acceleration 
except:
    raise Exception("\nPlease make sure you've configured Colab to request a GPU instance type.\n\n"
                    "At top of Colab, try: Runtime -> Change runtime type -> Hardware accelerator -> GPU -> Save\n")

# not allocated compatable GPU
if (my_gpu != b'Tesla T4') and (my_gpu != 'Tesla P100-PCIE...') and (my_gpu != 'GeForce GTX'):
    # allocated K80
    if my_gpu == 'Tesla K80':
        raise Exception("\nYou've been allocated a K80 instance\n\n"
                    "Unfortunately, this demo requires a T4 instance\n\n"
                    "At top of Colab, try: Runtime -> Reset all runtimes...\n")
    else:
        raise Exception(f"\nYou've achieved wizardy.\nyour GPU is {my_gpu}\nPlease inform info@blazingsql.com")

# allocated compatable GPU
else:
    print('Woo! You got the right kind of GPU!')

Woo! You got the right kind of GPU!


## Installs 

Below you will find three code blocks:
1. The first installs miniconda.
2. The second installs RAPIDS AI and sets up the system environment. 
3. The third installs BlazingSQL.

### Miniconda

In [2]:
# # intall miniconda
# !wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
# !chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
# !bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

### RAPIDS AI

In [3]:
# # install RAPIDS packages
# !conda install -q -y --prefix /usr/local -c nvidia -c rapidsai \
#   -c numba -c conda-forge -c pytorch -c defaults \
#   cudf=0.9 cuml=0.9 cugraph=0.9 python=3.6 cudatoolkit=10.0

# # set environment vars
# import sys, os, shutil
# sys.path.append('/usr/local/lib/python3.6/site-packages/')
# os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
# os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

# # copy .so files to current working dir
# for fn in ['libcudf.so', 'librmm.so']:
#     shutil.copy('/usr/local/lib/'+fn, os.getcwd())

### BlazingSQL

In [4]:
# # Install BlazingSQL for CUDA 10.0
# ! conda install -q -y --prefix /usr/local -c conda-forge -c defaults -c nvidia -c rapidsai \
#    -c blazingsql/label/cuda10.0 -c blazingsql \
#    blazingsql-calcite blazingsql-orchestrator blazingsql-ral blazingsql-python

# !pip install flatbuffers

# Import Packages

In [1]:
# Import RAPIDS AI stack
from blazingsql import BlazingContext
import cudf

bc = BlazingContext()

BlazingContext ready


# Download Data

In [2]:
# !wget https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_00.csv
# !wget https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_01.csv
# !wget https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_02.csv
# !wget https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_03.csv

# ETL: Read and Join CSVs


In [3]:
# set attribute column names 
column_names = ['key', 'fare_amount', 'pickup_longitude', 'pickup_latitude', 
                'dropoff_longitude', 'dropoff_latitude', 'passenger_count']
# and the type of each column
column_types = ['date64', 'float32', 'float32', 'float32', 
                'float32', 'float32', 'float32']

# load first csv 
gdf_00 = cudf.read_csv('taxi_00.csv', delimiter= ',', dtype = column_types, names = column_names)
# load second csv
gdf_01 = cudf.read_csv('taxi_01.csv', delimiter= ',', dtype = column_types, names = column_names)
# load third csv
gdf_02 = cudf.read_csv('taxi_01.csv', delimiter= ',', dtype = column_types, names = column_names)
# load fourth csv
gdf_03 = cudf.read_csv('taxi_01.csv', delimiter= ',', dtype = column_types, names = column_names)

# combine all those dataframes into one master dataframe
gdf = cudf.concat([gdf_00,gdf_01, gdf_02, gdf_03])

# what's it look like?
gdf.head()

,key,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2012-02-02 22:30:19.002,8.9,-73.988708,40.758804,-73.986519,40.737202,1.0
1,2014-09-20 07:19:24.001,4.0,-73.990204,40.746708,-73.994728,40.750515,1.0
2,2013-02-23 07:18:05.001,5.5,-74.016762,40.709438,-74.009003,40.719498,3.0
3,2015-04-18 23:49:27.009,13.5,-74.002708,40.733730,-73.986099,40.734776,1.0
4,2010-03-04 08:15:59.001,10.5,-73.988365,40.737663,-74.012459,40.713932,1.0


# ETL: Create Table

In [4]:
%time
# make a table from the combined df
bc.create_table('taxi', gdf)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.44 µs


# ETL: Query Tables for Training Data

In [5]:
# define the query
query = '''
        SELECT hour(key) as hours, month(key) as months, year(key) - 2000 as years,  
        dropoff_longitude - pickup_longitude as longitude_distance, 
        dropoff_latitude - pickup_latitude as latitude_distance, 
        passenger_count FROM main.taxi
        '''
# run query on table
X_train = bc.sql(query).get()

In [6]:
# extract dataframe
X_train_gdf = X_train.columns

# how's that look?
X_train_gdf.head()

,$f0,$f1,$f2,$f3,$f4,passenger_count
0,22,2,12,0.002190,-0.021603,1.0
1,7,9,14,-0.004524,0.003807,1.0
2,7,2,13,0.007759,0.010059,3.0
3,23,4,15,0.016609,0.001045,1.0
4,8,3,10,-0.024094,-0.023731,1.0


# Current status brief
- table is being made
  - but column names are not being coppied as expected 
- `X_train` is a whole thing 

In [7]:
# temp fix to columns not translating as expected
X_train_gdf.columns = ['hours', 'months', 'years', 
                       'longitude_distance', 'latitude_distance', 'passenger_count']

X_train_gdf.head()

,hours,months,years,longitude_distance,latitude_distance,passenger_count
0,22,2,12,0.002190,-0.021603,1.0
1,7,9,14,-0.004524,0.003807,1.0
2,7,2,13,0.007759,0.010059,3.0
3,23,4,15,0.016609,0.001045,1.0
4,8,3,10,-0.024094,-0.023731,1.0


In [8]:
X_train_gdf['longitude_distance'] = X_train_gdf['longitude_distance'].fillna(0).astype('float32')
X_train_gdf['latitude_distance'] = X_train_gdf['latitude_distance'].fillna(0).astype('float32')
X_train_gdf['passenger_count'] = X_train_gdf['passenger_count'].fillna(0).astype('float32')
X_train_gdf['months'] = X_train_gdf['months'].astype('float32') 
X_train_gdf['years'] = X_train_gdf['years'].astype('float32') 
X_train_gdf['hours'] = X_train_gdf['hours'].astype('float32')
        
X_train_gdf.head()

,hours,months,years,longitude_distance,latitude_distance,passenger_count
0,22.0,2.0,12.0,0.002190,-0.021603,1.0
1,7.0,9.0,14.0,-0.004524,0.003807,1.0
2,7.0,2.0,13.0,0.007759,0.010059,3.0
3,23.0,4.0,15.0,0.016609,0.001045,1.0
4,8.0,3.0,10.0,-0.024094,-0.023731,1.0


In [9]:
X_train_gdf.isnull().sum(), len(X_train_gdf)

(hours                 0
 months                0
 years                 0
 longitude_distance    0
 latitude_distance     0
 passenger_count       0
 dtype: int64, 20000000)

In [10]:
# query dependent variable y
y_train = bc.sql('SELECT fare_amount FROM main.taxi').get()
# extract dataframe
y_train_gdf = y_train.columns
# shrink to single column
# y_train_gdf = y_train_gdf['fare_amount']

y_train_gdf.head()

,fare_amount
0,8.9
1,4.0
2,5.5
3,13.5
4,10.5


# Install cuML on Colab

In [11]:
# !wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
# !chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
# !bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
# import sys
# sys.path.append('/usr/local/lib/python3.6/site-packages/')

In [12]:
# !conda install -c rapidsai -c nvidia -c conda-forge \
#     -c defaults cuml=0.10 python=3.7 cudatoolkit=10.0 -y

# Linear Regression: Train Model

In [17]:
%%time

import cuml
from cuml import LinearRegression

#create model
# lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = "eig")
lr = LinearRegression()

#train model
reg = lr.fit(X_train_gdf,y_train_gdf)

#print results
print("Coefficients:")
print(reg.coef_)
print(" ")
print(" Y intercept:")
print(reg.intercept_)

Exception ignored in: <finalize object at 0x7f555a4972e0; dead>
Traceback (most recent call last):
  File "/home/rodrigo/anaconda3/envs/rapidsenv/lib/python3.7/site-packages/numba/utils.py", line 669, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "/home/rodrigo/anaconda3/envs/rapidsenv/lib/python3.7/site-packages/rmm/rmm.py", line 220, in finalizer
    librmm.rmm_free(handle, stream)
  File "rmm/_lib/lib.pyx", line 229, in rmm._lib.lib.rmm_free
  File "rmm/_lib/lib.pyx", line 218, in rmm._lib.lib.c_free
  File "rmm/_lib/lib.pyx", line 49, in rmm._lib.lib.check_error
rmm.rmm.RMMError: RMM_ERROR_CUDA_ERROR


RuntimeError: Exception occured! file=/conda/conda-bld/libcuml_1571339163826/work/cpp/src_prims/common/cuml_allocator.hpp line=109: FAIL: call='cudaMalloc(&ptr, n)'. Reason:out of memory

Obtained 64 stack frames
#0 in /home/rodrigo/anaconda3/envs/rapidsenv/lib/python3.7/site-packages/cuml/common/../../../../libcuml++.so(_ZN8MLCommon9Exception16collectCallStackEv+0x3e) [0x7f546412983e]
#1 in /home/rodrigo/anaconda3/envs/rapidsenv/lib/python3.7/site-packages/cuml/common/../../../../libcuml++.so(_ZN8MLCommon9ExceptionC2ERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE+0x80) [0x7f546412a350]
#2 in /home/rodrigo/anaconda3/envs/rapidsenv/lib/python3.7/site-packages/cuml/common/../../../../libcuml++.so(_ZN8MLCommon22defaultDeviceAllocator8allocateEmP11CUstream_st+0x10e) [0x7f546412a63e]
#3 in /home/rodrigo/anaconda3/envs/rapidsenv/lib/python3.7/site-packages/cuml/common/../../../../libcuml++.so(_ZN8MLCommon11buffer_baseIfNS_15deviceAllocatorEEC2ESt10shared_ptrIS1_EP11CUstream_stm+0xa5)

# Linear Regression: Use Model to Predict Future Taxi Fares 

For this we are using a second dataset with data but no fare amount. We will predict fare_amounts. 

Here is a public link to that file: https://drive.google.com/file/d/1UG5-dXNPsAWZb0bJgquEg1qZsm12jzZW/view?usp=sharing

You will need to download that file and upload it to the colab. 

In [18]:
# Create Test Data Table

column_names = ['key', 'fare_amount', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count']
column_types = ['date64', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32']

# !wget 'https://blazingsql-demos.s3-us-west-1.amazonaws.com/test.csv'

gdf2 = cudf.read_csv('test.csv', delimiter= ',', dtype = column_types, names = column_names)

bc.create_table('test', gdf2)


In [20]:
# Query Test Data Table to Create GDF
X_test = bc.sql('SELECT hour(key) as hours, month(key) as months, year(key) - 2000 as years,  dropoff_longitude - pickup_longitude as longitude_distance, dropoff_latitude - pickup_latitude as latitude_distance , passenger_count FROM main.test').get()
X_test_gdf = X_test.columns

In [23]:
# temp fix to columns not translating as expected
X_test_gdf.columns = ['hours', 'months', 'years', 
                       'longitude_distance', 'latitude_distance', 'passenger_count']

X_test_gdf.head()

,hours,months,years,longitude_distance,latitude_distance,passenger_count
0,13,1,15,-0.008110,-0.019970,1.0
1,13,1,15,-0.012024,0.019814,1.0
2,11,10,11,0.002869,-0.005119,1.0
3,21,12,12,-0.009277,-0.016178,1.0
4,21,12,12,-0.022537,-0.045345,1.0


In [25]:
X_test_gdf['longitude_distance'] = X_test_gdf['longitude_distance'].fillna(0).astype('float32')
X_test_gdf['latitude_distance'] = X_test_gdf['latitude_distance'].fillna(0).astype('float32')
X_test_gdf['passenger_count'] = X_test_gdf['passenger_count'].fillna(0).astype('float32')

X_test_gdf['months'] = X_test_gdf['months'].astype('float32') 
X_test_gdf['years'] = X_test_gdf['years'].astype('float32') 
X_test_gdf['hours'] = X_test_gdf['hours'].astype('float32')

# this is the data we will use to predict future ride costs
X_test_gdf.head()

,hours,months,years,longitude_distance,latitude_distance,passenger_count
0,13.0,1.0,15.0,-0.008110,-0.019970,1.0
1,13.0,1.0,15.0,-0.012024,0.019814,1.0
2,11.0,10.0,11.0,0.002869,-0.005119,1.0
3,21.0,12.0,12.0,-0.009277,-0.016178,1.0
4,21.0,12.0,12.0,-0.022537,-0.045345,1.0


In [26]:
# Predict Fare Amounts 
predictions = lr.predict(X_test_gdf)

print(predictions)

TypeError: must be real number, not NoneType

In [27]:
#Combine into a table of table points and predictions
X_test_gdf['predicted_fare'] = predictions

print(X_test_gdf)

NameError: name 'predictions' is not defined

 ##  Predict Cost from Grand Central Station to Samsung Next NYC at 7:00 AM on May 15th, 2020

In [ ]:
samsung_ride = cudf.DataFrame([('hours', 7.0), ('days',15.0), ('months', 5.0), ('years', 20.0), ('longitude_distance', 0.012727), ('latitude_distance', 0.008484), ('passenger_count', 1.0)])

samsung_ride['hours'] = samsung_ride['hours'].astype('float32')
samsung_ride['days'] = samsung_ride['days'].astype('float32')
samsung_ride['months'] = samsung_ride['months'].astype('float32')
samsung_ride['years'] = samsung_ride['years'].astype('float32')
samsung_ride['longitude_distance'] = samsung_ride['longitude_distance'].astype('float32')
samsung_ride['latitude_distance'] = samsung_ride['latitude_distance'].astype('float32')
samsung_ride['passenger_count'] = samsung_ride['passenger_count'].astype('float32')

samsung_prediction = lr.predict(samsung_ride)
print(samsung_prediction)